In [1]:
# 디렉토리 관련
import os
import natsort
import os.path
import glob

# 작동 관련
import pandas as pd
import numpy as np
import json
import re

In [2]:
# 비디오 폴더들 전부 나열하기
path = 'C:/Users/admin/Desktop/CAT/RAW/*/**'
video_list = natsort.natsorted( glob.glob(path) )
video_list

['C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201028_cat-arch-000156.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201113_cat-arch-000157.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201114_cat-arch-000925.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201115_cat-arch-001607.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201118_cat-arch-000248.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201118_cat-arch-000458.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201120_cat-arch-001449.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201123_cat-arch-000924.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201125_cat-arch-000928.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-003285.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-003532.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-003817.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-004079.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-009771.mp4',
 'C:/U

In [3]:
len(video_list)

195

In [4]:
# json 파일들 전부 나열하기
path = 'C:/Users/admin/Desktop/CAT/LABEL/*/**'
json_list = natsort.natsorted( glob.glob(path) )
json_list

['C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201028_cat-arch-000156.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201113_cat-arch-000157.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201114_cat-arch-000925.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201115_cat-arch-001607.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201118_cat-arch-000248.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201118_cat-arch-000458.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201120_cat-arch-001449.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201123_cat-arch-000924.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201125_cat-arch-000928.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201202_cat-arch-003285.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201202_cat-arch-003532.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201202_cat-arch-003817.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201202_cat-arch-0040

In [5]:
len(json_list)

195

In [6]:
videolist = []
videoname = []
for item in video_list:
    newitem = item.replace('\\', '/')
    videolist.append(newitem)
    name = newitem.split('/')[-1]
    videoname.append(name)

videolist[:2]
# videoname

['C:/Users/admin/Desktop/CAT/RAW/ARCH/20201028_cat-arch-000156.mp4',
 'C:/Users/admin/Desktop/CAT/RAW/ARCH/20201113_cat-arch-000157.mp4']

In [7]:
jsonlist = []
jsonname = []
for item in json_list:
    newitem = item.replace('\\', '/')
    jsonlist.append(newitem)
    name = newitem.split('/')[-1]
    jsonname.append(name)

jsonlist[:2]
# jsonname

['C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201028_cat-arch-000156.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201113_cat-arch-000157.mp4.json']

In [8]:
# 프레임 넘버를 뽑아내기 위한 정규화 함수 미리 설정
def getFrameNumEx(imgcol):
    p = '_(\d{1,5})_'
    if re.search(p, imgcol):
        result = re.findall(p, imgcol)
        return int( result[0] )

In [9]:
# 빈 데이터프레임 만들어두기
empty_df = pd.DataFrame(columns=['image','frame','0_x','0_y','1_x','1_y','2_x','2_y','3_x','3_y','4_x','4_y','5_x','5_y','6_x','6_y','7_x','7_y','8_x','8_y','9_x','9_y','10_x','10_y','11_x','11_y','12_x','12_y','13_x','13_y','14_x','14_y'])

In [10]:
for i, (raw, label) in enumerate(zip(videolist, jsonlist)):
    df = pd.DataFrame()
    print(raw, label)
    # image 컬럼 : 이미지 목록 획득
    imgs = natsort.natsorted(glob.glob(f'{raw}/*.jpg'))
    foldername=[]
    for item in imgs:
        newitem = item.split('/')[-1].replace('\\', '/')
        foldername.append(newitem)
    df['image'] = foldername
    df['frame'] = list(map( getFrameNumEx, foldername ))
    # annotation 정보 추출    
    with open( label , encoding='utf-8') as f:
        metaInfo = json.load(f)
    # 실제 이미지가 존재하는 프레임만 추출하여 정보 획득
    check_idx = 0
    keyFrames = list()
    x_cords = list()
    y_cords = list()
    for anno in metaInfo['annotations']:
        if df['frame'][ check_idx ] == anno['frame_number']:
            keyFrames.append( anno['keypoints'] )
            # 세부 좌표 추출
            xs = list()
            ys = list()
            for n in range(1, 15+1):
                tmp = anno['keypoints'][ str(n) ]
                if tmp:
                    xs.append( tmp['x'] )
                    ys.append( tmp['y'] )
                else:
                    xs.append( 0 )
                    ys.append( 0 )   
            x_cords.append( xs )         
            y_cords.append( ys )
            check_idx += 1
            if df.shape[0] == check_idx:
                break
        else:
            continue
        #print( anno['frame_number'], anno['timestamp'] ) 
    #df['keyFrames'] = keyFrames
    df_x_cords = pd.DataFrame(x_cords, columns=[ f'{m}_x' for m in range(15) ])
    df_y_cords = pd.DataFrame(y_cords, columns=[ f'{m}_y' for m in range(15) ])
    # 샘플 1개만 진행, 아래를 제거하면 전체 데이터에 대해서 수행
    final_df = pd.concat( [df, df_x_cords, df_y_cords], axis=1 )
    empty_df = pd.concat( [empty_df, final_df])

C:/Users/admin/Desktop/CAT/RAW/ARCH/20201028_cat-arch-000156.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201028_cat-arch-000156.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201113_cat-arch-000157.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201113_cat-arch-000157.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201114_cat-arch-000925.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201114_cat-arch-000925.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201115_cat-arch-001607.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201115_cat-arch-001607.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201118_cat-arch-000248.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201118_cat-arch-000248.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201118_cat-arch-000458.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201118_cat-arch-000458.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201120_cat-arch-001449.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201120_cat-arch-001449.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201

In [12]:
empty_df.reset_index(drop=True, inplace=True)
empty_df.drop(columns=['frame'], inplace=True)
empty_df

,image,0_x,0_y,1_x,1_y,2_x,2_y,3_x,3_y,4_x,...,10_x,10_y,11_x,11_y,12_x,12_y,13_x,13_y,14_x,14_y
0,20201028_cat-arch-000156.mp4/frame_0_timestamp...,563,600,553,503,0,0,0,0,0,...,718,694,676,699,773,695,775,562,862,640
1,20201028_cat-arch-000156.mp4/frame_12_timestam...,551,528,548,450,568,546,552,544,0,...,718,696,680,703,767,725,782,568,866,636
2,20201028_cat-arch-000156.mp4/frame_15_timestam...,554,536,548,450,0,0,0,0,0,...,720,693,671,723,771,731,782,568,866,636
3,20201028_cat-arch-000156.mp4/frame_102_timesta...,559,1071,483,931,0,0,0,0,347,...,678,962,0,0,772,1021,696,655,882,445
4,20201028_cat-arch-000156.mp4/frame_105_timesta...,502,1016,435,888,0,0,0,0,295,...,610,879,0,0,754,959,607,594,866,530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,20201117_cat-armstretch-001283.mp4/frame_197_t...,184,282,269,162,266,356,184,344,0,...,500,596,0,0,517,692,0,0,712,612
4809,20201117_cat-armstretch-001283.mp4/frame_203_t...,162,219,247,107,235,297,156,276,0,...,0,0,267,855,464,834,0,0,709,614
4810,20201117_cat-armstretch-001283.mp4/frame_209_t...,157,121,230,8,210,199,159,173,0,...,0,0,0,0,457,838,0,0,709,614
4811,20201117_cat-armstretch-001283.mp4/frame_215_t...,181,74,0,0,225,148,176,128,412,...,406,757,244,881,465,863,0,0,728,600


In [13]:
# final_df = pd.concat( [df, df_x_cords, df_y_cords], axis=1 )
# final_df

In [14]:
empty_df.to_csv('./globfinal.csv')